<a href="https://colab.research.google.com/github/ggmeiner22/Estimating-Camera-Pose-from-a-Planar-Object/blob/main/EstimatingCameraPoseFromAPlanarObject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Goal

Given a calibrated camera (i.e., known intrinsics **K**) and a single image of a planar object, estimate the camera **pose** (i.e., extrinsics **R**, **t**).

I will:

1. build a Gradio UI to click 2D features and manage point order,
2. estimate pose **from a homography** (explicit derivation), and
3. estimate pose using **OpenCV** functions,
4. compare the two (short notes only).

Installs dependencies

In [ ]:
%pip -q install opencv-python numpy gradio matplotlib

Imports

In [ ]:
import json, io, math
import numpy as np
import cv2 as cv
import gradio as gr
import matplotlib.pyplot as plt

from matplotlib.figure import Figure

Load inrinsics from JSOn

In [ ]:
def load_intrinsics_from_json(json_bytes):
    d = json.loads(json_bytes.decode("utf-8"))
    K = np.array(d["K"], dtype=np.float64)
    dist = np.array(d.get("distCoeffs", [0,0,0,0,0]), dtype=np.float64).reshape(-1,1)
    return K, dist

Load 2D model points

In [2]:
def load_model_points_json(json_bytes):
    """
    Load model points from a JSON file with structure:
    {
      "units": "meters",
      "square_size_meters": 0.02177,
      "rows": 6,
      "cols": 9,
      "ordering": "row-major from (0,0)",
      "points": [
        { "X": 0.0, "Y": 0.0, "Z": 0.0 },
        ...
      ]
    }
    Returns: Nx3 numpy array [X,Y,Z]
    """
    import json, numpy as np

    # Parse JSON
    data = json.loads(json_bytes.decode("utf-8"))

    # Must have "points" array
    if "points" not in data:
        raise ValueError("JSON must have a 'points' array with X,Y,Z fields.")

    pts = data["points"]
    if not all("X" in p and "Y" in p for p in pts):
        raise ValueError("Each point must have X and Y (Z optional).")

    # Extract X,Y,Z into array
    X = [float(p["X"]) for p in pts]
    Y = [float(p["Y"]) for p in pts]
    Z = [float(p.get("Z", 0.0)) for p in pts]

    arr = np.column_stack([X, Y, Z]).astype(np.float64)

    # Optional: sort row-major if rows/cols exist
    if "rows" in data and "cols" in data:
        rows, cols = int(data["rows"]), int(data["cols"])
        if len(arr) == rows * cols:
            arr = arr.reshape(rows, cols, 3).reshape(-1, 3, order="C")

    return arr
